<a href="https://colab.research.google.com/github/matthewbhlau/crispr-kit/blob/main/dual_color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import math
from google.colab import files
from IPython.display import display, HTML


In [2]:
'''
This function allows users to upload files
'''
def upload_file(prompt):
    print(prompt)
    uploaded = files.upload()
    # The keys of 'uploaded' are the filenames of the uploaded files.
    # We assume here that exactly one file was uploaded.
    return next(iter(uploaded))

In [3]:
'''
This function takes in the 7 cropped images/conditions and calculates the mean
RGB values of the images. Returns a list with total rgb values (21 total)
'''
def rgb_analysis(*image_paths):
    total_rgb = []
    for i, image_path in enumerate(image_paths):
        ind_rgb = []
        # Load the image
        img = cv2.imread(image_path)
        # Split the image into its Red, Green, and Blue channels
        b, g, r = cv2.split(img)
        # Calculate the mean RGB values for each channel and add to list
        r_mean = np.mean(r)
        ind_rgb.append(r_mean)
        g_mean = np.mean(g)
        ind_rgb.append(g_mean)
        b_mean = np.mean(b)
        ind_rgb.append(b_mean)
        total_rgb.append(ind_rgb)
        #print(f"Image {i + 1} RGB values: R={r_mean}, G={g_mean}, B={b_mean}")
    print(total_rgb)
    return total_rgb

In [4]:
'''
This function takes in the total_rgb list and subtracts all values from those in the
NC condition (7th condition). Outputs list conditions with 21 values
'''
def scale(conditions):
  r_nc = conditions[6][0]
  g_nc = conditions[6][1]
  b_nc = conditions[6][2]
  for condition in conditions:
    condition[0] -= r_nc
    condition[1] -= g_nc
    condition[2] -= b_nc
  return(conditions)

In [5]:
'''
This function computes the first raw values for both the yellow and red conditions by dimensional reduction.
Outputs one list with two nested lists representing yellow and red, each with 7 values, one for each condition
'''
def raw_calc(conditions):
  # projection for dimensional reduction
  def calculate_reductions(index):
      r_pc, g_pc, b_pc = conditions[index]
      denom = math.sqrt(r_pc ** 2 + g_pc ** 2 + b_pc ** 2)
      return [(new[0] * r_pc + new[1] * g_pc + new[2] * b_pc) / denom for new in conditions]

  yellow_reductions = calculate_reductions(1)
  red_reductions = calculate_reductions(0)
  total_reductions = []
  total_reductions.append(yellow_reductions)
  total_reductions.append(red_reductions)
  return total_reductions

In [6]:
'''
This function first computes the yellow and red shifts and use these two values
to calculate the adjusted values. Finally we take the adjusted values and update
the list as a ratio of the base value. Outputs two lists, each with 5 values.
This list will be graphed out
'''
def adjust(total_reductions):
    # calculate yellow and red shifts
    yellow_shift = total_reductions[0][0]/total_reductions[1][0]
    red_shift = total_reductions[1][1]/total_reductions[0][1]
    #print(yellow_shift, red_shift)

    # calculate adjusted values
    y_original = total_reductions[0]
    r_original = total_reductions[1]
    y_new = [y_original[i] - yellow_shift * r_original[i] for i in range(len(y_original))]
    r_new = [r_original[i] - red_shift * y_original[i] for i in range(len(r_original))]

    # calculate ratios
    y_new = [val / y_new[1] for val in y_new]
    r_new = [val / r_new[0] for val in r_new]

    return y_new, r_new


In [7]:
'''
This function plots out the two lists in the form of a
bar chart (side by side yellow and red bars) for each of the 7 condition
'''

def expression_plot(y_new, r_new, filename):
    # positions for the bars
    bar_width = 0.2
    index = np.arange(len(y_new))

    # Creating the figure first
    fig, ax = plt.subplots(figsize=(15, 8))

    # creating the bars
    bar1 = ax.bar(index, y_new, bar_width, label="Yellow", color='yellow')
    bar2 = ax.bar(index + bar_width, r_new, bar_width, label="Red", color='red')

    # adding the blue line at y=0
    ax.axhline(0, color='blue')

   # adding labels and title and print out entire
    ax.set_xlabel('Conditions', fontsize=16)  # Adjust the font size
    ax.set_ylabel('Yellow and Red values', fontsize=16)  # Adjust the font size
    ax.set_title('Relative Expression', fontsize=20)  # Adjust the font size
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(('R','Y','R+Y','RT', 'YT', 'BT', 'NC'))
    ax.legend()

    conditions = ['R', 'Y', 'R+Y', 'RT', 'YT', 'BT', 'NC']
    for i in range(len(conditions)):
        print(f"For condition {conditions[i]}, Yellow value: {y_new[i]}, Red value: {r_new[i]}")

    plt.tight_layout()

    # Save the graph as a JPG file
    plt.savefig(filename, dpi=300)  # You can adjust the dpi as needed

    plt.show()



In [ ]:
import csv
def main():
    # Ask the user to upload the images.
    uploaded = files.upload()

    # The keys of 'uploaded' are the filenames of the uploaded files.
    image_files = list(uploaded.keys())

    # Sort files based on last modified time
    image_files.sort(key=os.path.getmtime)

    # Now 'image_files' contains the filenames of all the uploaded files, sorted by last modified time.
    total_rgb = rgb_analysis(*image_files)
    conditions = scale(total_rgb)
    values = raw_calc(conditions)
    final_a, final_b = adjust(values)
    display(HTML('<span style="color:red; font-size:30px;">PLEASE SCREENSHOT EVERYTHING BELOW HERE FOR SURVEY</span>'))

    # Plot and save the first graph as a JPG file
    expression_plot(final_a, final_b, filename = 'expression_plot.jpg')

    # Save calculated values to the first CSV file
    with open('calculated_values.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Condition', 'Yellow Value', 'Red Value'])  # Write header

        conditions = ['R', 'Y', 'R+Y', 'RT', 'YT', 'BT', 'NC']
        for i in range(len(conditions)):
            csv_writer.writerow([conditions[i], final_a[i], final_b[i]])

    # Create a new set of data by multiplying values by 100
    final_a_new = [val * 100 for val in final_a]
    final_b_new = [val * 100 for val in final_b]

    # Apply all conditions in a single list comprehension - make 100 if greater than 100; make 0 if less than 0
    final_a_new = [min(max(val, 0), 100) for val in final_a_new]
    final_b_new = [min(max(val, 0), 100) for val in final_b_new]

    # Plot and save the second graph as a JPG file
    expression_plot(final_a_new, final_b_new, filename = 'expression_plot_modified.jpg')

    # Save calculated values to the second CSV file
    with open('calculated_values_modified.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Condition', 'Yellow Value', 'Red Value'])  # Write header

        for i in range(len(conditions)):
            csv_writer.writerow([conditions[i], final_a_new[i], final_b_new[i]])

    # Download both CSV files and JPG files to your local machine's Downloads folder
    files.download('calculated_values.csv')
    files.download('calculated_values_modified.csv')
    files.download('expression_plot.jpg')
    files.download('expression_plot_modified.jpg')


if __name__ == "__main__":
    main()
